# Python to MLIR

## Step 1: Build a model in TensorFlow

In [1]:
import os
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.python.framework.convert_to_constants import convert_variables_to_constants_v2
import numpy as np

2024-10-31 02:42:17.058470: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-10-31 02:42:17.062368: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-10-31 02:42:17.117794: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-10-31 02:42:18.197328: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


### Lenet-5

In [2]:
# Define the LeNet-5 architecture according to the table provided
def lenet5_model():
    model = models.Sequential()

    # Input Layer: Image (32x32)
    
    # Layer 1: Convolutional Layer
    model.add(layers.Conv2D(6, (5, 5), strides=1, activation='tanh', input_shape=(32, 32, 1)))
    # Layer 2: Average Pooling Layer
    model.add(layers.AveragePooling2D(pool_size=(2, 2), strides=2))

    # Layer 3: Convolutional Layer
    model.add(layers.Conv2D(16, (5, 5), strides=1, activation='tanh'))
    # Layer 4: Average Pooling Layer
    model.add(layers.AveragePooling2D(pool_size=(2, 2), strides=2))

    # Layer 5: Convolutional Layer (Fully Connected Convolution)
    model.add(layers.Conv2D(120, (5, 5), strides=1, activation='tanh'))

    # Flatten the output for the fully connected layers
    model.add(layers.Flatten())

    # Layer 6: Fully Connected Layer
    model.add(layers.Dense(84, activation='tanh'))

    # Output Layer: Fully Connected Layer with 10 classes (softmax activation)
    model.add(layers.Dense(10, activation='softmax'))

    return model

# Create the model
model = lenet5_model()

# Compile the model
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

# Display the model summary
model.summary()



/home/dmanjun5/soda-opt/.soda/lib/python3.10/site-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 28, 28, 6)      │           156 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ average_pooling2d               │ (None, 14, 14, 6)      │             0 │
│ (AveragePooling2D)              │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 10, 10, 16)     │         2,416 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ average_pooling2d_1             │ (None, 5, 5, 16)       │             0 │
│ (AveragePooling2D)              │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 1, 1, 120)      │        48,120 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 120)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 84)             │        10,164 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 10)             │           850 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 61,706 (241.04 KB)

 Trainable params: 61,706 (241.04 KB)

 Non-trainable params: 0 (0.00 B)

### Resnet-18

In [ ]:
# Define the residual block
def residual_block(x, filters, stride=1, downsample=False):
    identity = x
    
    # First convolutional layer in the block
    x = layers.Conv2D(filters, kernel_size=3, strides=stride, padding='same', use_bias=False)(x)
    x = layers.BatchNormalization()(x)
    x = layers.ReLU()(x)
    
    # Second convolutional layer in the block
    x = layers.Conv2D(filters, kernel_size=3, strides=1, padding='same', use_bias=False)(x)
    x = layers.BatchNormalization()(x)
    
    # Downsample the input if required
    if downsample:
        identity = layers.Conv2D(filters, kernel_size=1, strides=stride, use_bias=False)(identity)
        identity = layers.BatchNormalization()(identity)
    
    # Add the identity (skip connection) to the output
    x = layers.Add()([x, identity])
    x = layers.ReLU()(x)
    
    return x

# Define the ResNet-18 model
def ResNet18(input_shape=(224, 224, 3), num_classes=1000):
    inputs = layers.Input(shape=input_shape)
    
    # Initial convolution and max pooling
    x = layers.Conv2D(64, kernel_size=7, strides=2, padding='same', use_bias=False)(inputs)
    x = layers.BatchNormalization()(x)
    x = layers.ReLU()(x)
    x = layers.MaxPooling2D(pool_size=3, strides=2, padding='same')(x)
    
    # Residual blocks
    x = residual_block(x, 64)
    x = residual_block(x, 64)
    
    x = residual_block(x, 128, stride=2, downsample=True)
    x = residual_block(x, 128)
    
    x = residual_block(x, 256, stride=2, downsample=True)
    x = residual_block(x, 256)
    
    x = residual_block(x, 512, stride=2, downsample=True)
    x = residual_block(x, 512)
    
    # Global average pooling and output layer
    x = layers.GlobalAveragePooling2D()(x)
    outputs = layers.Dense(num_classes, activation='softmax')(x)
    
    # Create the model
    model = models.Model(inputs, outputs)
    return model

# Instantiate the model
model = ResNet18(input_shape=(224, 224, 3), num_classes=1000)

# Print the model summary
model.summary()


### MobilenetV3-Small

In [ ]:
from tensorflow.keras.applications import MobileNetV3Small

# Load the MobileNetV3 Small model, pretrained on ImageNet, with the default top layers included
model = MobileNetV3Small(
    input_shape=(224, 224, 3),  # Standard ImageNet input size
    include_top=True,           # Includes the original classification layers
    weights='imagenet',         # Pretrained on ImageNet
    classes=1000                # 1000 output classes for ImageNet
)

# Compile the model
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Summary of the model
model.summary()

### MobilenetV3-Large

In [ ]:
from tensorflow.keras.applications import MobileNetV3Large

# Load the MobileNetV3 Small model, pretrained on ImageNet, with the default top layers included
model = MobileNetV3Large(
    input_shape=(224, 224, 3),  # Standard ImageNet input size
    include_top=True,           # Includes the original classification layers
    weights='imagenet',         # Pretrained on ImageNet
    classes=1000                # 1000 output classes for ImageNet
)

# Compile the model
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Summary of the model
model.summary()

### VGG-19

In [ ]:
from tensorflow.keras.applications import VGG19

# Load the VGG19 model pre-trained on ImageNet with the specified input shape
model = VGG19(weights='imagenet', include_top=True, input_shape=(224, 224, 3))

# Summary of the model
model.summary()

## Step 2: Convert model to protobuf

In [3]:
!mkdir -p python_to_mlir

In [4]:
save_path = os.path.join(os.getcwd(), "python_to_mlir/simple/")

# Save model to SavedModel format
tf.saved_model.save(model, save_path)

# Convert Keras model to ConcreteFunction
full_model = tf.function(lambda x: model(x))
full_model = full_model.get_concrete_function(
    x=[
        tf.TensorSpec(model.inputs[0].shape, model.inputs[0].dtype, name='x1')
    ])

# Get frozen ConcreteFunction
frozen_func = convert_variables_to_constants_v2(full_model)

# Save frozen graph from frozen ConcreteFunction to hard drive
tf.io.write_graph(graph_or_graph_def=frozen_func.graph,
                    logdir=os.getcwd(),
                    name="python_to_mlir/frozen_graph.pbtxt",
                    as_text=True)


INFO:tensorflow:Assets written to: /home/dmanjun5/soda-opt/thesis/DSE_final/python_to_mlir/simple/assets


INFO:tensorflow:Assets written to: /home/dmanjun5/soda-opt/thesis/DSE_final/python_to_mlir/simple/assets
2024-10-31 02:42:32.173184: I tensorflow/core/grappler/devices.cc:66] Number of eligible GPUs (core count >= 8, compute capability >= 0.0): 0
2024-10-31 02:42:32.173361: I tensorflow/core/grappler/clusters/single_machine.cc:361] Starting new session


'/home/dmanjun5/soda-opt/thesis/DSE_final/python_to_mlir/frozen_graph.pbtxt'

## Step 3: Transform protobuf into MLIR




In [5]:
!scripts/protobuf-to-tosa.sh python_to_mlir/frozen_graph.pbtxt python_to_mlir/tosa.mlir

2024-10-31 09:42:35.974427: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE3 SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


## Step 4: Lower MLIR to Linalg on Buffers

Remember to set the input shape correctly in [protobuf-to-tosa.sh](./scripts/protobuf-to-tosa.sh)

In [6]:
!scripts/tosa-to-linalg.sh python_to_mlir/tosa.mlir python_to_mlir/linalg-buffers.mlir

### Copy with appropriate name

As LeNet architecture is executed, the model is copied to models folder as [lenet.mlir](./models/lenet.mlir)

In [7]:
!cp python_to_mlir/linalg-buffers.mlir models/lenet.mlir